#### Initializing Dims and Facts (once only)
* Create empty table for facts. Dims will be added to faa.airport_dims
* Create dims stored procedures and execute
* Create facts stored procedures and execute  

#### Updating Dims and Facts with new data
* Run R script to prep data for staging area ([elmer_prep_faa_cargo.R](https://github.com/psrc/trends-airports/blob/master/elmer/elmer_prep_faa_cargo.R))
* Execute dims stored procedures
* Execute facts stored procedures

In [4]:
SELECT top 10 *
FROM Sandbox.Christy.faa_cargo;

In [13]:
SELECT *
FROM stg.faa_cargo;

# Dimensions

Create/alter dims stored procedure that adds on to faa.aiport_dims

In [14]:
ALTER PROCEDURE [faa].[merge_cargo_dim]
as

;with cte as (
	select ISNULL(RO, '--') AS RO, 
		ISNULL(ST, '--') AS ST, 
		ISNULL(Locid, '--') as Locid, 
		ISNULL(City, '--') as City, 
		ISNULL(Airportname, '--') as Airportname, 
		ISNULL(SL, '--') as SL, 
		ISNULL(Hub, '--') as Hub
	--from Sandbox.Christy.faa_cargo
    from stg.faa_cargo
	group by RO, ST, Locid, City, Airportname, SL, Hub
)
MERGE  faa.airport_dims as target
USING cte as source
ON (
	target.ro = source.RO
	and target.st = source.ST
	and target.loc_id = source.Locid
	and target.city_name = source.City
	and target.airport_name = source.Airportname
	and target.sl = source.SL
	and target.hub = source.Hub
)
WHEN NOT MATCHED THEN 
INSERT (ro, st, loc_id, city_name, airport_name, sl, hub)
VALUES (source.RO, 
	source.ST,
	source.Locid,
	source.City,
	source.Airportname,
	source.SL,
	source.Hub
);

Commands completed successfully.

Total execution time: 00:00:00.014

Run procedure

In [15]:
Exec faa.merge_cargo_dim;

(1 row affected)

Total execution time: 00:00:00.068

Check faa.airport_dim

In [16]:
SELECT *
FROM faa.airport_dims;

# Facts

Create Facts table

In [1]:
CREATE TABLE [faa].[cargo_facts](
	[cargo_fact_id] [int] IDENTITY(1,1) NOT NULL,
	[airport_dim_id] [int] NOT NULL,
	[data_year] [smallint] NOT NULL,
	[landed_weight] [float] NOT NULL,
 CONSTRAINT [pk_cargo_facts] PRIMARY KEY NONCLUSTERED 
(
	[airport_dim_id] ASC,
	[data_year] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY]
GO
CREATE CLUSTERED INDEX [idx_cargo_facts_clustered] ON [faa].[cargo_facts]
(
	[cargo_fact_id] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'cargo_facts' in the database.

Commands completed successfully.

Total execution time: 00:00:00.027

Check empty table

In [2]:
SELECT * 
FROM Elmer.faa.cargo_facts;

(0 rows affected)

Total execution time: 00:00:00.149

cargo_fact_id,airport_dim_id,data_year,landed_weight


#### Alter a stored procedure for merging FAA Cargo Facts

In [17]:
ALTER PROCEDURE [faa].[merge_cargo_facts] 
as

;with cte as (
	select ad.airport_dim_id, fe.landed_weight, fe.year
    from stg.faa_cargo as fe 
	--from Sandbox.Christy.faa_cargo as fe
        INNER JOIN Elmer.faa.airport_dims as ad
        ON fe.RO = ad.ro
        and fe.ST = ad.st
        and fe.Locid = ad.loc_id
        and fe.City = ad.city_name
        and fe.Airportname = ad.airport_name
        and fe.SL = ad.sl
        and fe.Hub = ad.hub
)
MERGE faa.cargo_facts as target
USING cte as source
ON (target.airport_dim_id = source.airport_dim_id
	and target.landed_weight = source.landed_weight
	and target.data_year = source.year
)
WHEN NOT MATCHED THEN 
INSERT (airport_dim_id, data_year, landed_weight)
VALUES (source.airport_dim_id,
    source.year, 
	source.landed_weight
);

Commands completed successfully.

Total execution time: 00:00:00.016

Run stored procedure

In [18]:
EXEC faa.merge_cargo_facts;

In [19]:
SELECT ad.*, cf.*
FROM faa.cargo_facts as cf 
JOIN faa.airport_dims as ad
ON cf.airport_dim_id = ad.airport_dim_id
WHERE city_name = 'Memphis';